This project will determine the best kind of movies for Microsoft to make.
First we will import the necessary packages, then we will load the web page to scrape. This page is a list of all-time grossing movies.

In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import sqlite3

In [2]:
html_page = requests.get('https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW')
soup = BeautifulSoup(html_page.content, 'html.parser') 

The table consists of four types of data: rank, title, lifetime gross, and year. Using the inspect element feature 
in the browser, it is evident that the rank is in mojo-field-type-rank; title in mojo-field-type-title, lifetime 
gross in mojo-field-type-rank, and release year in mojo-field-type-year

In [3]:
rank = soup.find('td', class_="mojo-field-type-rank")
rank 

<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">1</td>

We'll put the ranks of all the movies in a list.

In [4]:
rank_list = [r.text for r in soup.findAll('td', class_="mojo-field-type-rank")] 
print(len(rank_list), rank_list[:5])

200 ['1', '2', '3', '4', '5']


We'll then find the titles, gross earnings, and year released and put those in lists.

In [5]:
title = rank.nextSibling
title

<td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt4154796/?ref_=bo_cso_table_1">Avengers: Endgame</a></td>

In [6]:
title_list = [t.text for t in soup.findAll('td', class_="mojo-field-type-title")] 
print(len(title_list), title_list[:5])

200 ['Avengers: Endgame', 'Avatar', 'Titanic', 'Star Wars: Episode VII - The Force Awakens', 'Avengers: Infinity War']


In [7]:
money = title.nextSibling
money

<td class="a-text-right mojo-field-type-money">$2,797,800,564</td>

In [8]:
money_list = [m.text for m in soup.findAll('td', class_="mojo-field-type-money")] 
print(len(money_list), money_list[:5])

200 ['$2,797,800,564', '$2,790,439,092', '$2,471,754,307', '$2,068,454,310', '$2,048,359,754']


In [9]:
year = money.nextSibling
year

<td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/world/2019/?ref_=bo_cso_table_1">2019</a></td>

In [10]:
year_list = [y.text for y in soup.findAll('td', class_="mojo-field-type-year")] 
print(len(year_list), year_list[:5])

200 ['2019', '2009', '1997', '2015', '2018']


The links to the movie pages didn't have an attribute that was unique (it was the same for every link in the page). 
To get just the link, first I got a container with the url we need for each movie

In [11]:
link_list = [t for t in soup.findAll('td', class_="mojo-field-type-title")] 
print(len(link_list), link_list[0])

200 <td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt4154796/?ref_=bo_cso_table_1">Avengers: Endgame</a></td>


Then I extracted just the href from each.

In [12]:
new_link_list = []
for t in link_list:
    link = [x.get('href') for x in t.findAll('a', class_="a-link-normal")]
    link_str = link[0]
    new_link_list.append(link_str)
print(len(new_link_list), new_link_list[0])

200 /title/tt4154796/?ref_=bo_cso_table_1


I put the lists into a dataframe.

In [13]:
df = pd.DataFrame([rank_list, title_list, new_link_list, money_list, year_list]).transpose()
df.columns = ['Lifetime_Rank', 'Movie_Title', 'Movie_Page', 'Lifetime_Gross', 'Year_Released']
df

,Lifetime_Rank,Movie_Title,Movie_Page,Lifetime_Gross,Year_Released
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018
...,...,...,...,...,...
195,196,The Simpsons Movie,/title/tt0462538/?ref_=bo_cso_table_196,"$536,414,293",2007
196,197,The Revenant,/title/tt1663202/?ref_=bo_cso_table_197,"$532,950,503",2015
197,198,The Meg,/title/tt4779682/?ref_=bo_cso_table_198,"$530,259,473",2018
198,199,Ralph Breaks the Internet,/title/tt5848272/?ref_=bo_cso_table_199,"$529,323,962",2018


Now I repeated the process for the tables of top grossing films for 2019 and 2020.

In [14]:
# Get top grossing films, 2019, 2020
html_page2 = requests.get('https://www.boxofficemojo.com/year/2019/?ref_=bo_yl_table_3')
soup2 = BeautifulSoup(html_page2.content, 'html.parser') 
rank_2019 = soup2.find('td', class_="mojo-field-type-rank")
title_2019 = soup2.find('td', class_="mojo-field-type-release")
money_2019 = soup2.find('td', class_="mojo-field-type-money")
print(rank_2019, title_2019, money_2019)

<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column">1</td> <td class="a-text-left mojo-field-type-release mojo-cell-wide"><a class="a-link-normal" href="/release/rl3059975681/?ref_=bo_yld_table_1">Avengers: Endgame</a></td> <td class="a-text-right mojo-field-type-money hidden">-</td>


In [17]:
rank_2019_list = [r.text for r in soup2.findAll('td', class_="mojo-field-type-rank")] 
print(len(rank_2019_list), rank_2019_list[:5])
title_2019_list = [t.text for t in soup2.findAll('td', class_="mojo-field-type-release")] 
print(len(title_2019_list), title_2019_list[:5])
money_2019_list = [m.text for m in soup2.findAll('td', class_="mojo-field-type-money")] 
print(len(money_2019_list), money_2019_list[:5])

911 ['1', '2', '3', '4', '5']
911 ['Avengers: Endgame', 'The Lion King', 'Toy Story 4', 'Frozen II', 'Captain Marvel']
2733 ['-', '$858,373,000', '$858,373,000', '-', '$543,638,043']


In [18]:
revised_money_2019_list = []
for i in range(len(money_2019_list)):
    if i%3 == 1:
       revised_money_2019_list.append(money_2019_list[i])
print(len(revised_money_2019_list), revised_money_2019_list[:5])

911 ['$858,373,000', '$543,638,043', '$434,038,008', '$430,144,682', '$426,829,839']


In [19]:
# as in the first dataframe, I extracted the link to the page for each movie
link_list_2019 = [t for t in soup2.findAll('td', class_="mojo-field-type-release")] 
new_link_list_2019 = []
for t in link_list_2019:
    link = [x.get('href') for x in t.findAll('a', class_="a-link-normal")]
    link_str = link[0]
    new_link_list_2019.append(link_str)
print(len(new_link_list_2019), new_link_list_2019[0])

911 /release/rl3059975681/?ref_=bo_yld_table_1


In [20]:
df_2019 = pd.DataFrame([rank_2019_list, title_2019_list, new_link_list_2019, revised_money_2019_list]).transpose()
df_2019.columns = ['2019_Rank', 'Movie_Title', 'Movie_Page', '2019_Gross']
df_2019

,2019_Rank,Movie_Title,Movie_Page,2019_Gross
0,1,Avengers: Endgame,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000"
1,2,The Lion King,/release/rl3321923073/?ref_=bo_yld_table_2,"$543,638,043"
2,3,Toy Story 4,/release/rl3798500865/?ref_=bo_yld_table_3,"$434,038,008"
3,4,Frozen II,/release/rl2424210945/?ref_=bo_yld_table_4,"$430,144,682"
4,5,Captain Marvel,/release/rl3009644033/?ref_=bo_yld_table_5,"$426,829,839"
...,...,...,...,...
906,907,Henchmen,/release/rl1241941505/?ref_=bo_yld_table_907,$297
907,908,1945,/release/rl3506603521/?ref_=bo_yld_table_908,$236
908,909,Tall Tales from the Magical Garden of Antoon K...,/release/rl17401345/?ref_=bo_yld_table_909,$220
909,910,Over the Limit,/release/rl268928513/?ref_=bo_yld_table_910,$117


In [21]:
html_page3 = requests.get('https://www.boxofficemojo.com/year/2020/?grossesOption=calendarGrosses')
soup3 = BeautifulSoup(html_page3.content, 'html.parser') 

# retrieve the data for each column
rank_2020_list = [r.text for r in soup3.findAll('td', class_="mojo-field-type-rank")] 
title_2020_list = [t.text for t in soup3.findAll('td', class_="mojo-field-type-release")] 
money_2020_list = [m.text for m in soup3.findAll('td', class_="mojo-field-type-money")] 

# select only the money column of that year's gross
revised_money_2020_list = []
for i in range(len(money_2020_list)):
    if i%3 == 1:
       revised_money_2020_list.append(money_2020_list[i])
    
# as in the first dataframe, I extracted the link to the page for each movie
link_list_2020 = [t for t in soup3.findAll('td', class_="mojo-field-type-release")] 
new_link_list_2020 = []
for t in link_list_2020:
    link = [x.get('href') for x in t.findAll('a', class_="a-link-normal")]
    link_str = link[0]
    new_link_list_2020.append(link_str)
print(len(new_link_list_2020), new_link_list_2020[0])

# create the dataframe
df_2020 = pd.DataFrame([rank_2020_list, title_2020_list, new_link_list_2020, revised_money_2020_list]).transpose()
df_2020.columns = ['2020_Rank', 'Movie_Title', 'Movie_Page', '2020_Gross']
df_2020

454 /release/rl1182631425/?ref_=bo_yld_table_1


,2020_Rank,Movie_Title,Movie_Page,2020_Gross
0,1,Bad Boys for Life,/release/rl1182631425/?ref_=bo_yld_table_1,"$204,417,855"
1,2,1917,/release/rl2969994753/?ref_=bo_yld_table_2,"$157,901,466"
2,3,Sonic the Hedgehog,/release/rl4244997633/?ref_=bo_yld_table_3,"$146,066,470"
3,4,Jumanji: The Next Level,/release/rl755467777/?ref_=bo_yld_table_4,"$124,736,710"
4,5,Star Wars: Episode IX - The Rise of Skywalker,/release/rl3305145857/?ref_=bo_yld_table_5,"$124,496,308"
...,...,...,...,...
449,450,Asako I & II,/release/rl1359513089/?ref_=bo_yld_table_450,$231
450,451,Chained for Life,/release/rl1292600833/?ref_=bo_yld_table_451,$115
451,452,Shooting the Mafia,/release/rl2366342657/?ref_=bo_yld_table_452,$88
452,453,Benjamin the Elephant (2020),/release/rl1180795649/?ref_=bo_yld_table_453,$49


I limited my dataframes to the top 200 grossing movies of each year.

In [22]:
df_2020 = df_2020.drop(df_2020.index[200:])
df_2020

,2020_Rank,Movie_Title,Movie_Page,2020_Gross
0,1,Bad Boys for Life,/release/rl1182631425/?ref_=bo_yld_table_1,"$204,417,855"
1,2,1917,/release/rl2969994753/?ref_=bo_yld_table_2,"$157,901,466"
2,3,Sonic the Hedgehog,/release/rl4244997633/?ref_=bo_yld_table_3,"$146,066,470"
3,4,Jumanji: The Next Level,/release/rl755467777/?ref_=bo_yld_table_4,"$124,736,710"
4,5,Star Wars: Episode IX - The Rise of Skywalker,/release/rl3305145857/?ref_=bo_yld_table_5,"$124,496,308"
...,...,...,...,...
195,196,Extra Ordinary,/release/rl2562359809/?ref_=bo_yld_table_196,"$164,346"
196,197,Palm Springs,/release/rl3175187201/?ref_=bo_yld_table_197,"$164,000"
197,198,A Call to Spy,/release/rl3796140801/?ref_=bo_yld_table_198,"$158,864"
198,199,Love Story50th Anniversary,/release/rl934707713/?ref_=bo_yld_table_199,"$153,504"


In [23]:
df_2019 = df_2019.drop(df_2019.index[200:])
df_2019

,2019_Rank,Movie_Title,Movie_Page,2019_Gross
0,1,Avengers: Endgame,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000"
1,2,The Lion King,/release/rl3321923073/?ref_=bo_yld_table_2,"$543,638,043"
2,3,Toy Story 4,/release/rl3798500865/?ref_=bo_yld_table_3,"$434,038,008"
3,4,Frozen II,/release/rl2424210945/?ref_=bo_yld_table_4,"$430,144,682"
4,5,Captain Marvel,/release/rl3009644033/?ref_=bo_yld_table_5,"$426,829,839"
...,...,...,...,...
195,196,Welcome to Marwen,/release/rl2264958465/?ref_=bo_yld_table_196,"$2,405,625"
196,197,"My People, My Country",/release/rl2433451521/?ref_=bo_yld_table_197,"$2,356,683"
197,198,Promare,/release/rl3104540161/?ref_=bo_yld_table_198,"$2,287,950"
198,199,Super 30,/release/rl4211639809/?ref_=bo_yld_table_199,"$2,269,878"


Merging the three dataframes.

In [24]:
df_new = pd.merge(df, df_2019, on = "Movie_Title", how = "outer") 
len(df_new)

382

In [310]:
df_new2 = pd.merge(df_new, df_2020, on = "Movie_Title", how = "outer") 
len(df_new2)

551

Move the movie link data from the secondary and tertiary columns into the primary link column.

In [311]:
i = 0
for j in range(0, 551):
    if df_new2.isnull().Movie_Page_x[i]:
        df_new2.Movie_Page_x[i] = df_new2.Movie_Page[i]
    if df_new2.isnull().Movie_Page_x[i]:
        df_new2.Movie_Page_x[i] = df_new2.Movie_Page_y[i]
    i +=1
# Then count the NaN cells to make sure there are none
count = df_new2["Movie_Page_x"].isna().sum()
count

0

For the 2019 and 2020 ranked movies, filled in the year released in the Year_Released column.

In [312]:
for y in df_new2.index:
    if pd.isna(df_new2['Year_Released'][y]) == True and pd.isna(df_new2['2019_Rank'][y]) == False:
        df_new2.Year_Released[y] = 2019
        
for y in df_new2.index:
    if pd.isna(df_new2['Year_Released'][y]) == True and pd.isna(df_new2['2020_Rank'][y]) == False:
        df_new2.Year_Released[y] = 2020

Scrape the web site for the domestic gross, budget, MPAA rating and genres of each movie.

In [215]:
movie_url_list = []
gross_master_list = []
budget_master_list = []
mpaa_master_list = []
genres_master_list = []

j=0
    
for m in df_new2.Movie_Title:
    movie_url = 'https://www.boxofficemojo.com' + df_new2.Movie_Page_x[j]
#     movie_url_list.append(movie_url)
    html_page5 = requests.get(movie_url)
    soup5 = BeautifulSoup(html_page5.content, 'html.parser')
    
# the domestic gross from the first span tag
    gross = soup5.find('span', class_="money")
    gross = str(gross)
    gross = gross[20:-7]

# the budget is not the first span tag with the money class, so it took extra steps
    budget = soup5.find_all('span', text=re.compile('Budget')) 
    budget_parent = []
    budget_parent = [budget.parent for budget in budget]
    if not budget_parent:
        budget_money = [np.nan]
    else:
        budget_parent_string = budget_parent[0]
        budget_money = [m.text for m in budget_parent_string.findAll('span', class_="money")]
    budget_money_string = budget_money[0]

# likewise for MPAA rating
    mpaa = soup5.find_all('span', text=re.compile('MPAA'))
    mpaa_parent = []
    mpaa_parent = [mpaa.parent for mpaa in mpaa]
    if not mpaa_parent:
        mpaa_rating = [np.nan, np.nan]
    else:
        mpaa_parent_string = mpaa_parent[0]
        mpaa_rating = [m.text for m in mpaa_parent_string.findAll('span')]
    mpaa_rating_string = mpaa_rating[1]

# #likewise for genres
    genres = soup5.find_all('span', text=re.compile('Genres')) 
    genres_parent = []
    genres_parent = [genres.parent for genres in genres]
    if not genres_parent:
        genres_categories = [np.nan, np.nan]
    else:
        genres_parent_string = genres_parent[0]
        genres_categories = [m.text for m in genres_parent_string.findAll('span')]
    genres_categories_string = genres_categories[1]

    gross_master_list.append(gross)
    budget_master_list.append(budget_money_string)
    mpaa_master_list.append(mpaa_rating_string)
    genres_master_list.append(genres_categories_string)
    
    j += 1
    if j > 550:
        break
 

Check the length of the lists.

In [325]:
len(gross_master_list), len(budget_master_list), len(mpaa_master_list), len(genres_master_list)

(551, 551, 551, 551)

In [222]:
genres_master_list[0]

'Action\n    \n        Adventure\n    \n        Drama\n    \n        Sci-Fi'

Make the genre categories for each movie a list.

In [328]:
movie_genres_list = []
genres_master_list_split = []
for x in genres_master_list:
    if type(x) != float:
        movie_genres_list = list(x.split("\n    \n        "))
    else:
        movie_genres_list = []
    genres_master_list_split.append(movie_genres_list)
genres_master_list_split[:5]

[['Action', 'Adventure', 'Drama', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Drama', 'Romance'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi']]

Add lists of scraped data to the master dataframe.

In [330]:
df_new2['Domestic_Gross']= gross_master_list
df_new2['Budget'] = budget_master_list
df_new2['MPAA'] = mpaa_master_list
df_new2['Genres'] = genres_master_list_split
df_new2.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross,Domestic_Gross,Budget,MPAA,Genres
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN,"$858,373,000","$356,000,000",PG-13,"[Action, Adventure, Drama, Sci-Fi]"
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN,NaN,NaN,NaN,"$760,507,625","$237,000,000",PG-13,"[Action, Adventure, Fantasy, Sci-Fi]"
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN,NaN,NaN,NaN,"$659,363,944","$200,000,000",PG-13,"[Drama, Romance]"
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN,NaN,NaN,NaN,"$936,662,225","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN,NaN,NaN,NaN,"$678,815,482",NaN,PG-13,"[Action, Adventure, Sci-Fi]"


In [ ]:
Deleted unnecessary columns to streamline the database and prevent complications when saving to sqlite

In [334]:
del df_new2['2019_Rank']
del df_new2['2019_Gross']
del df_new2['2020_Rank']
del df_new2['2020_Gross']
del df_new2['Movie_Page']
del df_new2['Movie_Page_y']
del df_new2['Lifetime_Gross']
del df_new2['Lifetime_Rank']

df_new2.tail()

,Movie_Title,Movie_Page_x,Year_Released,Domestic_Gross,Budget,MPAA,Genres
546,Extra Ordinary,/release/rl2562359809/?ref_=bo_yld_table_196,2020,"$164,346",NaN,R,"[Comedy, Fantasy, Horror]"
547,Palm Springs,/release/rl3175187201/?ref_=bo_yld_table_197,2020,"$164,000",NaN,R,"[Comedy, Fantasy, Mystery, Romance]"
548,A Call to Spy,/release/rl3796140801/?ref_=bo_yld_table_198,2020,"$159,014",NaN,PG-13,"[Biography, Crime, Drama, Thriller, War]"
549,Love Story50th Anniversary,/release/rl934707713/?ref_=bo_yld_table_199,2020,"$153,504",NaN,PG,"[Drama, Romance]"
550,The Dark and the Wicked,/release/rl4015883009/?ref_=bo_yld_table_200,2020,"$156,552",NaN,NaN,[Horror]


Put the scraped data in a dataframe so that it can be saved as a sqlite db. I don't want to do the scraping over and rehit the web site unnecessarily.

In [255]:
df_scraped_data = pd.DataFrame(list(zip(gross_master_list, budget_master_list, mpaa_master_list, genres_master_list)), 
               columns =['Domestic_Gross', 'Budget', 'MPAA', 'Genres']) 
df_scraped_data.head()

,Domestic_Gross,Budget,MPAA,Genres
0,"$858,373,000","$356,000,000",PG-13,Action\n \n Adventure\n \n ...
1,"$760,507,625","$237,000,000",PG-13,Action\n \n Adventure\n \n ...
2,"$659,363,944","$200,000,000",PG-13,Drama\n \n Romance
3,"$936,662,225","$245,000,000",PG-13,Action\n \n Adventure\n \n ...
4,"$678,815,482",NaN,PG-13,Action\n \n Adventure\n \n ...


Create a sqlite db to and put the scraped data in it.

In [256]:
conn = sqlite3.connect('scraped_data2.db') 
c = conn.cursor()

c.execute('CREATE TABLE scraped_data2 (Domestic_Gross, Budget, MPAA, Genres)')
conn.commit()

df_scraped_data.to_sql('scraped_data2', conn, if_exists='replace', index = False)


In [258]:
cnx = sqlite3.connect('scraped_data2.db')

df_imported = pd.read_sql_query("SELECT * FROM scraped_data2", cnx)
df_imported.head()

,Domestic_Gross,Budget,MPAA,Genres
0,"$858,373,000","$356,000,000",PG-13,Action\n \n Adventure\n \n ...
1,"$760,507,625","$237,000,000",PG-13,Action\n \n Adventure\n \n ...
2,"$659,363,944","$200,000,000",PG-13,Drama\n \n Romance
3,"$936,662,225","$245,000,000",PG-13,Action\n \n Adventure\n \n ...
4,"$678,815,482",None,PG-13,Action\n \n Adventure\n \n ...


Added the year released for those movies scraped from the 2019 and 2020 highest grossing lists.

In [338]:
# conn = sqlite3.connect('movie_data2.db') 
# c = conn.cursor()

# c.execute('CREATE TABLE movie_data (Lifetime_Rank, Movie_Title, Movie_Page_x, Lifetime_Gross, Year_Released, Movie_Page_y, Movie_Page, Domestic_Gross, Budget, MPAA, Genres)')
# conn.commit()

# df_new2.to_sql('movie_data2', conn, if_exists='replace', index = False)



In [290]:
# cnx = sqlite3.connect('movie_data2.db')

# df_imported2 = pd.read_sql_query("SELECT * FROM movie_data2", cnx)
# df_imported2.head(10)

,Domestic_Gross,Budget,MPAA,Genres
0,"$858,373,000","$356,000,000",PG-13,Action\n \n Adventure\n \n ...
1,"$760,507,625","$237,000,000",PG-13,Action\n \n Adventure\n \n ...
2,"$659,363,944","$200,000,000",PG-13,Drama\n \n Romance
3,"$936,662,225","$245,000,000",PG-13,Action\n \n Adventure\n \n ...
4,"$678,815,482",None,PG-13,Action\n \n Adventure\n \n ...
5,"$652,385,625","$150,000,000",PG-13,Action\n \n Adventure\n \n ...
6,"$543,638,043","$260,000,000",PG,Adventure\n \n Animation\n \n ...
7,"$623,357,910","$220,000,000",PG-13,Action\n \n Adventure\n \n ...
8,"$353,007,020","$190,000,000",PG-13,Action\n \n Adventure\n \n ...
9,"$477,373,578","$150,000,000",PG,Adventure\n \n Animation\n \n ...


In [339]:
df_new2['Domestic_Gross'][:5]

0    $858,373,000
1    $760,507,625
2    $659,363,944
3    $936,662,225
4    $678,815,482
Name: Domestic_Gross, dtype: object

Converted strings in Domestic Gross and Budget columns to integers, removing the commas and dollar signs.

In [433]:
i = 0
for y in df_new2:
    df_new2['Domestic_Gross'][i] = df_new2['Domestic_Gross'][i].replace(',', '')
    df_new2['Domestic_Gross'][i] = df_new2['Domestic_Gross'][i].replace('$', '')
    i +=1

In [518]:
i = 0
# for y in df_new2:
#     if type(df_new2['Budget'][i]) != float:
#         df_new2['Budget'][i] = df_new2['Budget'][i].replace(',', '')
#         df_new2['Budget'][i] = df_new2['Budget'][i].replace('$', '')
#     i += 1


KeyError: 560

Checked the data.

In [519]:
j = 0
for y in df_new3.index:
        print(j, df_new3['Domestic_Gross'][y], df_new3['Budget'][y])
        j += 1

0 858373000 356000000
1 760507625 237000000
2 659363944 200000000
3 936662225 245000000
4 678815482 nan
5 652385625 150000000
6 543638043 260000000
7 422783777 45000000
8 623357910 220000000
9 353007020 190000000
10 477373578 150000000
11 459005868 250000000
12 700426566 nan
13 381409310 nan
14 620181382 317000000
15 417719760 170000000
16 400953009 150000000
17 504481165 160000000
18 608581744 nan
19 226008385 250000000
20 409013994 200000000
21 336045770 74000000
22 408084349 250000000
23 335061807 nan
24 377845905 94000000
25 390532085 160000000
26 426829839 160000000
27 352390543 195000000
28 304360277 200000000
29 173956935 200000000
30 245439076 210000000
31 448139099 250000000
32 335451311 55000000
33 515202542 275000000
34 434038008 200000000
35 415004880 200000000
36 423315812 225000000
37 532177324 200000000
38 355559216 183000000
39 241071802 250000000
40 264624300 80000000
41 404214720 63000000
42 486295561 nan
43 474544677 115000000
44 334191110 200000000
45 341268248 nan


Converted Domestic Gross and Budget to numeric datatypes.

In [888]:
df_new4[["Domestic_Gross", "Budget"]] = df_new4[["Domestic_Gross", "Budget"]].apply(pd.to_numeric)
df_new4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 0 to 550
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Lifetime_Rank   200 non-null    object 
 1   Movie_Title     551 non-null    object 
 2   Movie_Page_x    551 non-null    object 
 3   Lifetime_Gross  200 non-null    object 
 4   Year_Released   551 non-null    object 
 5   2019_Rank       201 non-null    object 
 6   Movie_Page_y    201 non-null    object 
 7   2019_Gross      201 non-null    object 
 8   2020_Rank       200 non-null    object 
 9   Movie_Page      200 non-null    object 
 10  2020_Gross      200 non-null    object 
 11  Domestic_Gross  550 non-null    float64
 12  Budget          551 non-null    int64  
 13  MPAA            551 non-null    object 
 14  Genres          551 non-null    object 
dtypes: float64(1), int64(1), object(13)
memory usage: 88.9+ KB


In [903]:
i = 0
for x in df_new3.index:
    if df_new3['MPAA'][i] == 'PG-13':
        print(df_new3['Movie_Title'][i])
    i +=1
    
# print(df_new3['Movie_Title'].where(df_new3['MPAA'] == 'PG-13') )

Avengers: Endgame
Avatar
Titanic
Star Wars: Episode VII - The Force Awakens
Avengers: Infinity War
The Lion King
The Avengers
Frozen II
Avengers: Age of Ultron
Black Panther
Harry Potter and the Deathly Hallows: Part 2
Star Wars: Episode VIII - The Last Jedi
Incredibles 2
The Fate of the Furious
Minions
Captain America: Civil War
Aquaman
The Lord of the Rings: The Return of the King
Spider-Man: Far from Home
Captain Marvel
Transformers: Dark of the Moon
Skyfall
Fast & Furious Presents: Hobbs & Shaw
The Dark Knight Rises
Star Wars: Episode IX - The Rise of Skywalker
Pirates of the Caribbean: Dead Man's Chest
Rogue One: A Star Wars Story
Pirates of the Caribbean: On Stranger Tides
Jurassic Park
The Hobbit: An Unexpected Journey
The Dark Knight
Harry Potter and the Deathly Hallows: Part 1
Jumanji: Welcome to the Jungle
Pirates of the Caribbean: At World's End
The Hobbit: The Desolation of Smaug
The Hobbit: The Battle of the Five Armies
The Lord of the Rings: The Two Towers
Harry Potter an

In [304]:
# print(df_new3['Movie_Title'].where(df_new3['Domestic_Gross'] >1000000))


In [887]:
df_new4[["Domestic_Gross", "Budget"]] = df_new4[["Domestic_Gross", "Budget"]].apply(pd.to_numeric)
df_new4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 0 to 550
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Lifetime_Rank   200 non-null    object 
 1   Movie_Title     551 non-null    object 
 2   Movie_Page_x    551 non-null    object 
 3   Lifetime_Gross  200 non-null    object 
 4   Year_Released   551 non-null    object 
 5   2019_Rank       201 non-null    object 
 6   Movie_Page_y    201 non-null    object 
 7   2019_Gross      201 non-null    object 
 8   2020_Rank       200 non-null    object 
 9   Movie_Page      200 non-null    object 
 10  2020_Gross      200 non-null    object 
 11  Domestic_Gross  550 non-null    float64
 12  Budget          551 non-null    int64  
 13  MPAA            551 non-null    object 
 14  Genres          551 non-null    object 
dtypes: float64(1), int64(1), object(13)
memory usage: 88.9+ KB


In [39]:
# gross_master_list = []
# budget_master_list = []
# mpaa_master_list = []
# genres_master_list = []

# i = 0
# for x in df_imported.index:
#     gross_master_list.append(df_imported['Domestic_Gross'][i])
#     i +=1
 
# i = 0
# for x in df_imported.index:
#     budget_master_list.append(df_imported['Budget'][i])
#     i +=1
# i = 0
# for x in df_imported.index:
#     mpaa_master_list.append(df_imported['MPAA'][i])
#     i +=1
i = 0
for x in df_imported.index:
    genres_master_list.append(df_imported['Genres'][i])
    i +=1
len(gross_master_list), len(budget_master_list), len(mpaa_master_list), len(genres_master_list)

# df_new3 = df_new2
# df_new3['Domestic_Gross']= gross_master_list
# df_new3['Budget'] = budget_master_list
# df_new3['MPAA'] = mpaa_master_list
# df_new3['Genres'] = genres_master_list_split
# df_new3.head()

(550, 550, 550, 550)